<a href="https://colab.research.google.com/github/d4w17/desal/blob/main/Comprehensive_Water_Chemistry_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# Removed pyEQL import as it's no longer used for calculations

# Dictionary mapping ion symbols to their molar masses (g/mol)
molar_masses = {
    'Na+': 22.99, 'Cl-': 35.45, 'Mg+2': 24.31, 'K+': 39.10, 'SO4-2': 96.06, 'HCO3-': 61.02
}

# Dictionary mapping ion symbols to their charges
ion_charges = {
    'Na+': 1, 'Cl-': -1, 'Mg+2': 2, 'K+': 1, 'SO4-2': -2, 'HCO3-': -1
}


def calculate_total_molality(ion_concentrations):
    """Calculates total molality (approx. molarity) from a dictionary of ion concentrations (mg/L)."""
    total_molality = 0
    for ion, conc_mg_l in ion_concentrations.items():
        if ion in molar_masses:
            molar_mass = molar_masses[ion]
            # Convert mg/L to mol/L (approx. molality assuming density ~ 1 kg/L)
            moles_per_liter = (conc_mg_l / 1000.0) / molar_mass
            total_molality += moles_per_liter
        else:
            print(f"Warning: Molar mass for ion '{ion}' not found. Skipping for total molality calculation.")
    return total_molality


def calculate_osmotic_pressure(ion_concentrations):
    """Calculates osmotic pressure in bars using the van't Hoff equation from ion concentrations."""
    total_molality = calculate_total_molality(ion_concentrations)
    R = 0.08314  # L·bar/K·mol
    T = 298.15   # Kelvin (25 C)
    # The van't Hoff equation uses molality, but we are using mol/L approximation
    osmotic_pressure = total_molality * R * T
    return osmotic_pressure

def calculate_vapor_pressure_depression(ion_concentrations):
    """Calculates vapor pressure depression based on Raoult's Law from ion concentrations."""
    total_moles_solute = calculate_total_molality(ion_concentrations) # Total molality is approximately total moles/L
    moles_water = 1000 / 18.015  # moles in 1 kg of water (assuming density 1 kg/L)
    # Avoid division by zero if total_moles_solute + moles_water is zero
    if (total_moles_solute + moles_water) == 0:
        return np.nan
    mole_fraction_solute = total_moles_solute / (total_moles_solute + moles_water)
    P0_water = 0.03169  # bar at 25°C
    vpd = mole_fraction_solute * P0_water
    return vpd

def calculate_boiling_point_elevation(ion_concentrations):
    """Calculates boiling point elevation from ion concentrations."""
    total_molality = calculate_total_molality(ion_concentrations)
    Kb = 0.512  # °C/m for water
    bpe = Kb * total_molality
    return bpe

def calculate_ionic_strength_from_concentrations(ion_concentrations):
    """Calculates ionic strength (mol/L) directly from initial ion concentrations."""
    ionic_strength = 0
    for ion, conc_mg_l in ion_concentrations.items():
        if ion in molar_masses and ion in ion_charges:
            molar_mass = molar_masses[ion]
            charge = ion_charges[ion]
            moles_per_liter = (conc_mg_l / 1000.0) / molar_mass
            ionic_strength += moles_per_liter * charge**2
        else:
            print(f"Warning: Molar mass or charge for ion '{ion}' not found. Skipping for ionic strength calculation.")
    return 0.5 * ionic_strength


def main():
    """
    Main function to perform all calculations and generate plots.
    """
    print("--- Starting Water Chemistry Analysis ---")

    # --- 1. Data Setup ---
    samples_data = {
        'SP1': {'Na+': 45050, 'Cl-': 83811, 'Mg+2': 1017, 'K+': 1475, 'SO4-2': 194, 'HCO3-': 0},
        'SP2': {'Na+': 37949, 'Cl-': 81381, 'Mg+2': 1002, 'K+': 754,  'SO4-2': 562, 'HCO3-': 132.83},
        'SP3': {'Na+': 39429, 'Cl-': 72561, 'Mg+2': 599,  'K+': 262,  'SO4-2': 555, 'HCO3-': 532.60},
        'SP4': {'Na+': 43374, 'Cl-': 78314, 'Mg+2': 643,  'K+': 1285, 'SO4-2': 212, 'HCO3-': 0},
        'SP5': {'Na+': 39283, 'Cl-': 75198, 'Mg+2': 629,  'K+': 369,  'SO4-2': 638, 'HCO3-': 297.89},
        'Seawater': {'Na+': 10556, 'Cl-': 18980, 'Mg+2': 1272, 'K+': 380,  'SO4-2': 2649, 'HCO3-': 140.00}
    }

    results = []

    # --- 2. Perform Calculations for Each Sample ---
    print("Calculating properties for each sample...")
    for name, concentrations in samples_data.items():
        # --- Major Ions Scenario ---
        op_major = calculate_osmotic_pressure(concentrations) # Calculate directly
        vpd_major = calculate_vapor_pressure_depression(concentrations) # Calculate directly
        is_major = calculate_ionic_strength_from_concentrations(concentrations) # Calculate directly
        bpe_major = calculate_boiling_point_elevation(concentrations) # Calculate directly
        # Convert bar -> kJ/L -> kWh/m3
        work_major = (op_major * 0.1) / 3.6

        # --- Pure NaCl Scenario ---
        nacl_concentrations = {'Na+': concentrations.get('Na+', 0), 'Cl-': concentrations.get('Cl-', 0)}
        op_nacl = calculate_osmotic_pressure(nacl_concentrations) # Calculate directly
        vpd_nacl = calculate_vapor_pressure_depression(nacl_concentrations) # Calculate directly
        is_nacl = calculate_ionic_strength_from_concentrations(nacl_concentrations) # Calculate directly
        bpe_nacl = calculate_boiling_point_elevation(nacl_concentrations) # Calculate directly
        work_nacl = (op_nacl * 0.1) / 3.6


        results.append({
            'Sample': name,
            'OP_Major_bar': op_major, 'OP_NaCl_bar': op_nacl,
            'VPD_Major_bar': vpd_major, 'VPD_NaCl_bar': vpd_nacl,
            'IS_Major_mol_L': is_major, 'IS_NaCl_mol_L': is_nacl,
            'BPE_Major_C': bpe_major, 'BPE_NaCl_C': bpe_nacl,
            'Work_Major_kWh_m3': work_major, 'Work_NaCl_kWh_m3': work_nacl
        })

    results_df = pd.DataFrame(results)

    # --- 3. Display Consolidated Data Table ---
    print("\n--- Consolidated Results Table ---")
    print(results_df.round(2).to_string())
    print("\n" + "="*80 + "\n")


    # --- 4. Generate Plots ---
    print("Generating and saving plots...")

    # Plot 1: Osmotic Pressure
    plt.figure(figsize=(10, 6))
    plt.scatter(results_df['Sample'], results_df['OP_Major_bar'], label='Major Ions', marker='o', s=60)
    plt.scatter(results_df['Sample'], results_df['OP_NaCl_bar'], label='Pure NaCl', marker='x', s=60)
    plt.title('Osmotic Pressure vs. Sample Points', fontsize=16)
    plt.ylabel('Osmotic Pressure (bar)', fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('osmotic_pressure.png')
    plt.close()

    # Plot 2: Vapor Pressure Depression
    plt.figure(figsize=(10, 6))
    plt.scatter(results_df['Sample'], results_df['VPD_Major_bar'], label='Major Ions', marker='o', s=60)
    plt.scatter(results_df['Sample'], results_df['VPD_NaCl_bar'], label='Pure NaCl', marker='x', s=60)
    plt.title('Vapor Pressure Depression vs. Sample Points', fontsize=16)
    plt.ylabel('Vapor Pressure Depression (bar)', fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('vapor_pressure_depression.png')
    plt.close()

    # Plot 3: Ionic Strength
    plt.figure(figsize=(10, 6))
    plt.scatter(results_df['Sample'], results_df['IS_Major_mol_L'], label='Major Ions', marker='o', s=60)
    plt.scatter(results_df['Sample'], results_df['IS_NaCl_mol_L'], label='Pure NaCl', marker='x', s=60)
    plt.title('Ionic Strength vs. Sample Points', fontsize=16)
    plt.ylabel('Ionic Strength (mol/L)', fontsize=12)
    plt.legend()
    plt.grid(True, which='both', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('ionic_strength.png')
    plt.close()

    # Plot 4: Boiling Point Elevation
    plt.figure(figsize=(10, 6))
    plt.scatter(results_df['Sample'], results_df['BPE_Major_C'], label='Major Ions', marker='o', s=60)
    plt.scatter(results_df['Sample'], results_df['BPE_NaCl_C'], label='Pure NaCl', marker='x', s=60)
    plt.title('Boiling Point Elevation vs. Sample Points', fontsize=16)
    plt.ylabel('Boiling Point Elevation (°C)', fontsize=12)
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig('boiling_point_elevation.png')
    plt.close()

    # Plot 5: Least Work vs. Recovery
    fig, ax1 = plt.subplots(figsize=(12, 8))
    colors = plt.cm.viridis(np.linspace(0, 1, len(results_df)))
    r_plot = np.linspace(0.001, 0.85, 200)

    for index, row in results_df.iterrows():
        # Only plot if Work_Major_kWh_m3 is not NaN
        if not np.isnan(row['Work_Major_kWh_m3']):
            work_major_ions = row['Work_Major_kWh_m3'] / (1 - r_plot)
            ax1.plot(r_plot * 100, work_major_ions, label=f"{row['Sample']} (Major Ions)", color=colors[index], linestyle='-')
        # Only plot if Work_NaCl_kWh_m3 is not NaN
        if not np.isnan(row['Work_NaCl_kWh_m3']):
            work_nacl = row['Work_NaCl_kWh_m3'] / (1 - r_plot)
            ax1.plot(r_plot * 100, work_nacl, label=f"{row['Sample']} (Pure NaCl)", color=colors[index], linestyle='--')


    ax1.set_xlabel('Water Recovery (%)', fontsize=12)
    ax1.set_ylabel('Theoretical Minimum Work (kWh/m³)', fontsize=12)
    ax1.set_xlim(0, 85)
    ax1.set_ylim(0, 22)
    ax1.grid(True, which='both', linestyle='--', linewidth=0.5)
    ax1.legend(bbox_to_anchor=(1.17, 1), loc='upper left')

    ax2 = ax1.twiny()
    ax2.set_xlabel('Concentration Factor', fontsize=12)
    cf_labels = np.array([1, 1.5, 2, 3, 4, 5, 6])
    r_tick_positions = (1 - 1/cf_labels) * 100
    ax2.set_xticks(r_tick_positions)
    ax2.set_xticklabels([f"{val:.1f}" for val in cf_labels])
    ax2.set_xlim(ax1.get_xlim())

    fig.suptitle('Theoretical Minimum Work vs. Water Recovery and Concentration Factor', fontsize=16)
    fig.tight_layout(rect=[0, 0, 0.83, 0.95])
    plt.savefig('work_vs_recovery.png')
    plt.close()

    print("--- Analysis Complete ---")
    print("Generated 5 plot images: osmotic_pressure.png, vapor_pressure_depression.png, ionic_strength.png, boiling_point_elevation.png, work_vs_recovery.png")


if __name__ == '__main__':
    main()

--- Starting Water Chemistry Analysis ---
Calculating properties for each sample...

--- Consolidated Results Table ---
     Sample  OP_Major_bar  OP_NaCl_bar  VPD_Major_bar  VPD_NaCl_bar  IS_Major_mol_L  IS_NaCl_mol_L  BPE_Major_C  BPE_NaCl_C  Work_Major_kWh_m3  Work_NaCl_kWh_m3
0       SP1        109.20       107.18            0.0           0.0            2.27           2.16         2.26        2.21               3.03              2.98
1       SP2         99.52        97.82            0.0           0.0            2.08           1.97         2.06        2.02               2.76              2.72
2       SP3         94.39        93.25            0.0           0.0            1.95           1.88         1.95        1.93               2.62              2.59
3       SP4        103.05       101.53            0.0           0.0            2.12           2.05         2.13        2.10               2.86              2.82
4       SP5         96.10        94.94            0.0           0.0        

In [21]:
!pip install pyEQL